In [1]:
# B
# import packges
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
import nltk
from nltk.corpus import stopwords
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity
import time

import missingno as msno
import math

english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']


In [7]:
filename = 'data/prep.csv'
df = pd.read_csv(filename, index_col='Unnamed: 0')
df.head()
df = df.sample(n = 1000)
# read df
df_a = df[['asin','reviewText']]



In [ ]:
start = time.time()

# apply tokenization

df_a['tokens'] =  df_a["reviewText"].apply(word_tokenize)

# apply remove stopwords 
df_a["removeStopwords"] = df_a["tokens"].apply(lambda x: ([word for word in x if word not in (stopwords.words())]))

# remove punkt

df_a["removePunk"] = df_a["removeStopwords"].apply(lambda x: ([word for word in x if word not in (english_punctuations)]))

# lower case
df_a["lowStr"] = df_a["removePunk"].apply(lambda x: ([word.lower() for word in x]))

#Lemmatization

wnl = nltk.WordNetLemmatizer()
df_a["lem"] = df_a["lowStr"].apply(lambda x: ([wnl.lemmatize(word) for word in x]))

# clean df
df_b = df_a[['asin', 'lem']]
df_b["lem"] = df_b["lem"].apply(lambda x: ','.join(map(str, x)))

print(time.time() - start)

In [9]:
# functions 
def create_corpus(df):
    corpus = []
    for i in df:
        corpus.append(i)
    return corpus

# calculate cosine_similarity
def cs(i,j):
  doc1 = df.iloc[i]/df.max(axis=0)
  doc2 = df.iloc[j]/df.max(axis=0)

  docAB = doc1.mul(doc2, axis=0)
  numerator = docAB.sum() # inner prodoct

  powdf = pd.DataFrame([2]*col)
  powdf = powdf.T
  doc1square = doc1.pow(powdf)
  doc2square = doc2.pow(powdf)
  sum1 = doc1square.sum(axis = 1)
  sum2 = doc2square.sum(axis = 1)
  sqrtsum1 = math.sqrt(sum1)
  sqrtsum2 = math.sqrt(sum2)
  denominator = sqrtsum1 * sqrtsum2 # product of A vec and B vec
  if denominator == 0:
    cosSim = 0
  else:
    cosSim= numerator/denominator
  # print('cosSim(doc',i+1,',','doc',j+1,') is')
  # print(" ") 
  return cosSim


In [10]:
# df_a["tokens"]
df_a.head()

,asin,reviewText,tokens,removeStopwords,removePunk,lowStr,lem
60592,B006YBXZLK,big knife for those big knife jobs.\nsheath ra...,"[big, knife, for, those, big, knife, jobs, ., ...","[big, knife, big, knife, jobs, ., sheath, ratt...","[big, knife, big, knife, jobs, sheath, rattles...","[big, knife, big, knife, jobs, sheath, rattles...","[big, knife, big, knife, job, sheath, rattle, ..."
20967,B00005RCQS,Great mat wide and quiet,"[Great, mat, wide, and, quiet]","[Great, wide, quiet]","[Great, wide, quiet]","[great, wide, quiet]","[great, wide, quiet]"
624028,B00364E0C6,"Love the color, and get compliments on it all ...","[Love, the, color, ,, and, get, compliments, o...","[Love, color, ,, compliments, time, ., It, 's,...","[Love, color, compliments, time, It, 's, perfe...","[love, color, compliments, time, it, 's, perfe...","[love, color, compliment, time, it, 's, perfec..."
465719,B0058YXDH0,Great pants- Note that I am a 33-34 waist (but...,"[Great, pants-, Note, that, I, am, a, 33-34, w...","[Great, pants-, Note, I, 33-34, waist, (, wear...","[Great, pants-, Note, I, 33-34, waist, wearing...","[great, pants-, note, i, 33-34, waist, wearing...","[great, pants-, note, i, 33-34, waist, wearing..."
179324,B001NGD4UI,"These panniers are nice and large, perfect for...","[These, panniers, are, nice, and, large, ,, pe...","[These, panniers, nice, large, ,, perfect, com...","[These, panniers, nice, large, perfect, commut...","[these, panniers, nice, large, perfect, commut...","[these, pannier, nice, large, perfect, commute..."


In [11]:
df_a.shape

(1000, 7)

In [12]:
df_b.head()

,asin,lem
60592,B006YBXZLK,"big,knife,big,knife,job,sheath,rattle,secure,t..."
20967,B00005RCQS,"great,wide,quiet"
624028,B00364E0C6,"love,color,compliment,time,it,'s,perfect,teal,..."
465719,B0058YXDH0,"great,pants-,note,i,33-34,waist,wearing,36,-,c..."
179324,B001NGD4UI,"these,pannier,nice,large,perfect,commute,work,..."


In [55]:
corpusA = create_corpus(df_b["lem"])

In [56]:
corpusA

["big,knife,big,knife,job,sheath,rattle,secure,this,place,camp,axe,machete,held,i,'ve,you,big,knife,spending",
 'great,wide,quiet',
 "love,color,compliment,time,it,'s,perfect,teal,pictured,my,1,year,daughter,love,``,crabby,'',wear,house,regular,occasion,she,small,20lbs,n't,fit,but,work,n't,water",
 'great,pants-,note,i,33-34,waist,wearing,36,-,cut,small,compared,noted,only,minor,negative,point,bit,wrinkled,i,wish,side,zipper,pocket,that,extremely,comfortable,fairly,tough,wish,brown,color',
 'these,pannier,nice,large,perfect,commute,work,they,unclip,easily,bike,bike,rack,...',
 'great,product',
 'great,quality,work,awesome',
 'nice,sign',
 "work,great,n't,leak,1,year,old,love,drink",
 "all,dvd,'s,awesome,i,wish,mix,music,bit,the,music,move,exhilarate,concert,mix",
 'exceptionally,made,came,expected',
 'very,sharp,misjudged,size,loop,side,hoping,belt',
 '',
 'this,item,work,pretty,bit,distance,scope,you,scope,time,hit,i,wish,bit,larger,size,i,worked,could/should,distance,shooting,i,recom

In [57]:
tfidf = TfidfVectorizer()
tfs = tfidf.fit_transform(corpusA)

from sklearn.metrics.pairwise import cosine_similarity #Cosine sim calc
similarities = cosine_similarity(tfs)


In [58]:
print(similarities)

[[1.         0.         0.         ... 0.05916793 0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.03695732 0.         0.        ]
 ...
 [0.05916793 0.         0.03695732 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [59]:

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpusA)
Xarr = X.toarray()
Xarr



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [60]:
featurearr = vectorizer.get_feature_names_out()
featurearr = featurearr.tolist()

df = pd.DataFrame(data=Xarr)
df = pd.DataFrame(data=Xarr, columns=featurearr, index=df_b['asin'])
df


,00,09,10,100,1000,100lbs,105lbs,11,110,1159,...,yr,zero,zip,ziplock,zipped,zipper,zipping,zippo,zoom,zoomers
asin,,,,,,,,,,,,,,,,,,,,,
B006YBXZLK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00005RCQS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B00364E0C6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0058YXDH0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
B001NGD4UI,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00LQHOWUK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B001GFFRP6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
B0044UJF6C,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
row, col = df.shape

In [62]:


result = []
for i in range(0,1000):
  result.append(cs(0,i))

print(result)

[1.205676020408163, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.027598417595133595, 0.0, 0, 0.0, 0.0, 0.0028861501272920303, 0.0, 0.0, 0.35355339059327373, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0036076876591150378, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0, 0.0, 0.05892556509887895, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0.0017006802721088435, 0.0, 0.0, 0.0, 0.03658563566975077, 0.0, 0.0, 0.0, 0.0, 0.00835191380976326, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0014150515209827273, 0.0025510204081632647, 0.0, 0.004941058844013093, 0.0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0.0, 0.006378879538497859, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015624999999999997, 0.0, 0.09128709291752767, 0.0, 0, 0.0, 0.0392837100659193, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.20412414523193148, 0.044194173824159216, 0.0, 0, 0.0, 0, 0.0, 0, 0.0, 0.0, 0.0, 0, 0.0, 0.0, 0.00574864680399233, 0.0, 0.00294566463872258, 0.0, 0.0, 0.08333333333333331, 0.0

In [63]:
len(result)


1000

In [85]:
df_result = pd.DataFrame(data=result,columns=['CosSim'], index=df_b['asin'])
df_result = df_result[df_result.CosSim > 0]
df_result = df_result.sort_values(by=['CosSim'],ascending=False)

In [86]:
df_result


,CosSim
asin,
B006YBXZLK,1.205676
B0013R6JJ0,0.353553
B00B4D6WH2,0.242397
B001H53Q6M,0.237494
B00367P3TC,0.204124
...,...
B001CZ4ZN0,0.001415
B005LC57K0,0.001389
B001EWNEC4,0.001203


In [87]:
df_result =df_result.reset_index()
df_result = df_result.groupby(by=['asin']).sum()
df_result.sort_values('CosSim', ascending=False)

,CosSim
asin,
B006YBXZLK,1.205676
B00B4D6WH2,0.377227
B0013R6JJ0,0.353553
B001H53Q6M,0.261996
B00367P3TC,0.204124
...,...
B001CZ4ZN0,0.001415
B005LC57K0,0.001389
B001EWNEC4,0.001203


In [88]:
df_result =df_result.reset_index()
df_result['asin'].nunique()

192